<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/models/studio_1/GMB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Parte nuova testing


In [1]:
!pip install h5py
!pip install transformers

     |████████████████████████████████| 2.0MB 18.1MB/s 
     |████████████████████████████████| 3.2MB 45.4MB/s 
     |████████████████████████████████| 890kB 51.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=328fa83ba28fb4853a374695ded9b5b9c150d27bb7897faa69bae8983531cef7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import tensorflow as tf
import pandas as pd
import transformers
from transformers import BertTokenizer, TFBertModel, BertConfig
from tensorflow.keras import layers
from tensorflow import keras
from sklearn import preprocessing
from tokenizers import BertWordPieceTokenizer
import numpy as np

In [4]:
data_original = "/content/drive/Shareddrives/Deep Learning/datasets/GMB/GMB_dataset.tsv"
data_sinonimi = "/content/drive/Shareddrives/Deep Learning/datasets/GMB/GMB_sinonimi.csv"
data_embedding = "/content/drive/Shareddrives/Deep Learning/datasets/GMB/GMB_embedding.csv"
max_len = 384

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False, reduction=tf.keras.losses.Reduction.SUM
)

# Ignoring loss that is calculated due to padded targets
def masked_ce_loss(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 17))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def create_model(num_tags):
    # BERT Base model
    encoder = TFBertModel.from_pretrained("bert-base-uncased")

    # NER Model

    # Instantiate Keras tensors
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    # TFBertmodel
    embedding = encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
    # Add drop out layer
    embedding = layers.Dropout(0.5)(embedding)
    # Add softmax layer for classifying
    tag_logits = layers.Dense(num_tags+1, activation='softmax')(embedding)
    
    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[tag_logits],
    )
    optimizer = keras.optimizers.Adam(lr=3e-5)
    model.compile(optimizer=optimizer, loss=masked_ce_loss, metrics=['accuracy'])
    return model

In [6]:
#Reference - https://keras.io/examples/nlp/text_extraction_with_bert/
# Save the slow pretrained tokenizer
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
save_path = "bert_base_uncased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("bert_base_uncased/vocab.txt", lowercase=True)

MUST USE GPU or model dont load 

In [8]:
parz = pd.read_csv(data_original,sep='\t', encoding="latin-1", index_col=[0])

num_tags = parz["Tag"].nunique()


use_tpu = None
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    use_tpu = True
except:
    use_tpu = False

if use_tpu:
    # Create distribution strategy
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        new_model = create_model(num_tags)
else:
    new_model = create_model(num_tags)
    
new_model.summary()

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 384)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_1 (TFBertModel)   TFBaseModelOutputWit 109482240   input_4[0][0]                    
                                                                 input_6[0][0]                

In [ ]:
new_model.load_weights("/content/drive/Shareddrives/Deep Learning/datasets/GMB/test.h5")

###Train the model


In [10]:
# Get the sentences
def process_csv(data_path):
    df = pd.read_csv(data_path, encoding="latin-1", na_filter = False)
    df.loc[:, "Sentence #"] = df["Sentence #"].fillna(method="ffill")
    enc_tag = preprocessing.LabelEncoder()
    df.loc[:, "Tag"] = enc_tag.fit_transform(df["Tag"])
    sentences = df.groupby("Sentence #")["Word"].apply(list).values
    tag = df.groupby("Sentence #")["Tag"].apply(list).values
    return sentences, tag, enc_tag

# Get the sentences
def process_tsv(data_path):
    df =  pd.read_csv(data_path,sep='\t', encoding="latin-1", index_col=[0])
    df.loc[:, "Sentence #"] = df["Sentence #"].fillna(method="ffill")
    enc_tag = preprocessing.LabelEncoder()
    df.loc[:, "Tag"] = enc_tag.fit_transform(df["Tag"])
    sentences = df.groupby("Sentence #")["Word"].apply(list).values
    tag = df.groupby("Sentence #")["Tag"].apply(list).values
    return sentences, tag, enc_tag

# Prepare the inputs for feeding into model
def create_inputs_targets(data):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "tags": []
    }
    _,ext=os.path.splitext(data)
    if (ext== '.tsv'):
      sentences, tags, tag_encoder = process_tsv(data)
    else:
      sentences, tags, tag_encoder = process_csv(data)
    
    for sentence, tag in zip(sentences, tags):
        
        input_ids = []
        target_tags = []
        for idx, word in enumerate(sentence):
            ids = tokenizer.encode(word, add_special_tokens=False)
            input_ids.extend(ids.ids)
            num_tokens = len(ids)
            target_tags.extend([tag[idx]] * num_tokens)
                    
        
        # Pad truncate
        input_ids = input_ids[:max_len - 2]
        target_tags = target_tags[:max_len - 2]

        # Add [CLS] and [SEP]
        input_ids = [101] + input_ids + [102]
        target_tags = [16] + target_tags + [16]
        # token_type_ids does not matter as the task has only one sentence
        token_type_ids = [0] * len(input_ids)
        # Adding attention mask for non-padded input
        attention_mask = [1] * len(input_ids)
        
        # Making all the input enbedding same lenting by padding
        padding_len = max_len - len(input_ids)
        input_ids = input_ids + ([0] * padding_len)
        attention_mask = attention_mask + ([0] * padding_len)
        token_type_ids = token_type_ids + ([0] * padding_len)
        target_tags = target_tags + ([17] * padding_len)
        
        dataset_dict["input_ids"].append(input_ids)
        dataset_dict["token_type_ids"].append(token_type_ids)
        dataset_dict["attention_mask"].append(attention_mask)
        dataset_dict["tags"].append(target_tags)
        assert len(target_tags) == max_len, f'{len(input_ids)}, {len(target_tags)}'
                
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    # Creating array of input embeddings
    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = dataset_dict["tags"]
    return x, y, tag_encoder

merge original and sinonimi 


In [18]:
df1 = pd.read_csv(data_original,sep='\t', encoding="latin-1", index_col=[0])
df2 = pd.read_csv(data_sinonimi, encoding="latin-1", na_filter = False)
frames = [df1, df2]
result = pd.concat(frames)
result.to_csv("/content/drive/Shareddrives/Deep Learning/datasets/GMB/GMB_original_sinonimi.csv")

In [19]:
data_original_sinonimi = "/content/drive/Shareddrives/Deep Learning/datasets/GMB/GMB_original_sinonimi.csv"

In [20]:
x_train, y_train, tag_encoder = create_inputs_targets(data_original_sinonimi)

In [25]:
x_train

[array([[  101,  5190,  1997, ...,     0,     0,     0],
        [  101,  2945,  1997, ...,     0,     0,     0],
        [  101,  2027,  9847, ...,     0,     0,     0],
        ...,
        [  101,  1996,  4054, ...,     0,     0,     0],
        [  101, 16375, 13642, ...,     0,     0,     0],
        [  101,  1996,  4748, ...,     0,     0,     0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])]

In [ ]:
x_train, y_train, tag_encoder = create_inputs_targets(data)

bs = 64 if use_tpu else 16

model.fit(
    x_train,
    y_train,
    epochs=1,
    verbose=1,
    batch_size=bs,
    validation_split=0.1
)

###Evaluate the model

Original

In [ ]:
#crea i dati per essere elborati dal modello
x_test, y_test, tag_encoder = create_inputs_targets(data_original)
pred_test = new_model.evaluate(x_test,y_test,batch_size=16)
#definisce il dizioanrio dell'associazione classi valore numerico

188/188 [==============================] - 157s 808ms/step - loss: 7.9210 - accuracy: 0.9948


In [ ]:
pred_test

[8.508015632629395, 0.9944764375686646]

Sinonimi

In [ ]:
#crea i dati per essere elborati dal modello
x_test, y_test, tag_encoder = create_inputs_targets(data_sinonimi)
pred_test = new_model.evaluate(x_test,y_test,batch_size=16)
#definisce il dizioanrio dell'associazione classi valore numerico

188/188 [==============================] - 152s 807ms/step - loss: 20.6279 - accuracy: 0.9896


In [ ]:
pred_test

[20.62790298461914, 0.9896250367164612]

Embedding

In [ ]:
#crea i dati per essere elborati dal modello
x_test, y_test, tag_encoder = create_inputs_targets(data_embedding)
pred_test = new_model.evaluate(x_test,y_test,batch_size=16)
#definisce il dizioanrio dell'associazione classi valore numerico

188/188 [==============================] - 152s 809ms/step - loss: 16.2044 - accuracy: 0.9908


In [ ]:
pred_test

[16.204362869262695, 0.9908446073532104]